In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from time import time

In [2]:
df=pd.read_csv('train.csv')
df.info()

C:\Users\50259\AppData\Local\Temp\ipykernel_15624\3068579679.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('train.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [3]:
# Se determina columnas que tienen nulos menor al 5% para poder realizar imputacion de datos
cols_rescatables=[col for col in df.columns if (df[col].isnull().mean()<=0.05)]
# Se quitan duplicados
df.drop_duplicates(inplace=True)
# Se crea dataset 
df_temp=df[cols_rescatables]

In [4]:
# CONVERSION DE CATEGORICAS A NUMEROS 

# SE DETERMINA CARACTERES EXTRAÑOS QUE HAN DE SER REEMPLAZADOS PARA LOGRAR LA CONVERSION A NUMERO

# En este caso 'numero_' debe ser 'numero' por tanto se hace el cambio '_' = ''
df_temp['Annual_Income'] = df['Annual_Income'].str.replace('_', '')
df_temp['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace('_', '')
df_temp['Age'] = df['Age'].str.replace('_', '')
df_temp['Num_of_Loan'] = df['Num_of_Loan'].str.replace('_', '')

# En este caso '_' debe ser '0' porque significa sin cambio en limite de credito, por tanto se reemplaza '_' = '0'
df_temp['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].str.replace('_', '0')

# En este caso solo hay un numero con problema: ' __10000__' por tanto se sustituye con 10000
df_temp['Amount_invested_monthly'] = df['Amount_invested_monthly'].str.replace('__10000__', '10000')
# En este caso solo hay un numero con problema: _-3333333_ por tanto se sustituye con -3333333
df_temp['Monthly_Balance'] = df['Monthly_Balance'].str.replace('__-333333333333333333333333333__', '33333333333333333')


# CONVERTIR A NUMERICO 

# TIPO FLOAT

df_temp['Annual_Income'] = df_temp['Annual_Income'].astype(float)
df_temp['Changed_Credit_Limit'] = df_temp['Changed_Credit_Limit'].astype(float)
df_temp['Outstanding_Debt'] = df_temp['Outstanding_Debt'].astype(float)
df_temp['Amount_invested_monthly'] = df_temp['Amount_invested_monthly'].astype(float)
df_temp['Monthly_Balance'] = df_temp['Monthly_Balance'].astype(float)

# TIPO INT

df_temp['Age'] = df_temp['Age'].astype(int)
df_temp['Num_of_Loan'] = df_temp['Num_of_Loan'].astype(int)

C:\Users\50259\AppData\Local\Temp\ipykernel_15624\3529412718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Annual_Income'] = df['Annual_Income'].str.replace('_', '')
C:\Users\50259\AppData\Local\Temp\ipykernel_15624\3529412718.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace('_', '')
C:\Users\50259\AppData\Local\Temp\ipykernel_15624\3529412718.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [5]:
# SE VERIFICA:
df_temp.describe()

,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,100000.000000,1.000000e+05,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000,100000.000000,95521.000000,9.713200e+04
mean,110.649700,1.764157e+05,17.091280,22.47443,72.466040,3.009960,21.068780,10.171791,27.754251,1426.220376,32.285173,1403.118217,637.412998,3.088580e+12
std,686.244717,1.429618e+06,117.404834,129.05741,466.422621,62.647879,14.860104,6.880628,193.177339,1155.129026,5.116875,8306.041270,2043.319327,3.208492e+14
min,-500.000000,7.005930e+03,-1.000000,0.00000,1.000000,-100.000000,-5.000000,-6.490000,0.000000,0.230000,20.000000,0.000000,0.000000,7.759665e-03
25%,24.000000,1.945750e+04,3.000000,4.00000,8.000000,1.000000,10.000000,4.970000,3.000000,566.072500,28.052567,30.306660,74.534002,2.700037e+02
50%,33.000000,3.757861e+04,6.000000,5.00000,13.000000,3.000000,18.000000,9.250000,6.000000,1166.155000,32.305784,69.249473,135.925682,3.364770e+02
75%,42.000000,7.279092e+04,7.000000,7.00000,20.000000,5.000000,28.000000,14.660000,9.000000,1945.962500,36.496663,161.224249,265.731733,4.696852e+02
max,8698.000000,2.419806e+07,1798.000000,1499.00000,5797.000000,1496.000000,67.000000,36.970000,2597.000000,4998.070000,50.000000,82331.000000,10000.000000,3.333333e+16


In [6]:
# SE CREA LISTADO (CLASIFICACION DE VARIABLES)

catGen=[col for col in df_temp.columns if (df_temp[col].dtypes=='object')]
numGen=[col for col in df_temp.columns if ((df_temp[col].dtypes!='object'))]

In [7]:
# BALANCEO DE VARIABLE CATEGORICA

df_standard = df_temp[df_temp['Credit_Score'] == 'Standard']
df_poor = df_temp[df_temp['Credit_Score'] == 'Poor']
df_good = df_temp[df_temp['Credit_Score'] == 'Good']
cantidadstandard = 2*df_good.shape[0]
df_standard = df_standard.sample(n=cantidadstandard, random_state=2023)
df_temp = pd.concat([df_standard, df_poor, df_good])
df_temp.shape

(82482, 23)

In [8]:
# IMPUTACION VARIABLE CATEGORICA

# Se elimina faltantes de comportamiento de pago debido a que son particulares a cada cliente. CCA
df_temp = df_temp[df_temp['Payment_Behaviour'].str.contains('!@9#%8') == False]
# Se desconoce Ocupacion de persona asi que se coloca como otra
df_temp['Occupation'] = df_temp['Occupation'].str.replace('_______', 'Other')
# Se coloca desconocido al mix de credito pues es un comportamiento muy particular y puede tratarse de personas recien ingresadas
df_temp['Credit_Mix'] = df_temp['Credit_Mix'].str.replace('_', 'Unknown')


In [9]:
# IMPUTACION VARIABLE NUMERICA

# Imputacion con la media
df_temp['Num_Credit_Inquiries']=np.round(df_temp['Num_Credit_Inquiries'].fillna(df_temp['Num_Credit_Inquiries'].mean()))
df_temp['Monthly_Balance']=np.round(df_temp['Monthly_Balance'].fillna(df_temp['Monthly_Balance'].mean()))

# Imputacion con la mediana
df_temp['Amount_invested_monthly']=np.round(df_temp['Amount_invested_monthly'].fillna(df_temp['Amount_invested_monthly'].median()))

In [10]:
# TRATAMIENTO DE OUTLIERS

def detectOutliersLimits (dataset,col):
# Descripcion: calcula limites superiores e inferiores para identificar outliers
# input: dataset->dataframe pandas
# output: tupla de floats con los limites
    IQR=dataset[col].quantile(0.75)-dataset[col].quantile(0.25)
    LI=dataset[col].quantile(0.25)-(IQR*1.75)
    LS=dataset[col].quantile(0.75)+(IQR*1.75)

    return LI,LS


def Capping(data,colist):
# Descripcion: convierte outliers en limites superiores e inferiores
# input: data->dataframe pandas, colist-> arreglo con el nombre de las columnas
# output: transforma el dataframe ingresado en las columnas especificadas
    for c in colist:
        LI,LS=detectOutliersLimits(data,c)
        if '_code' in c:
            data[c]=data[c]
        else:
            data[c]=np.where(data[c]>LS,LS,
                                    np.where(data[c]<LI,LI,data[c]))

continuas=['Annual_Income', 'Interest_Rate','Delay_from_due_date','Changed_Credit_Limit','Outstanding_Debt','Credit_Utilization_Ratio',
            'Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance','Num_Bank_Accounts','Num_Credit_Card','Num_Credit_Inquiries']

Capping(df_temp,continuas)

In [11]:
# Eliminando valores menores a 0 para transformacion de variables continuas

for col in continuas:
    minimo=df_temp[col].min()
    if minimo<=0:
        df_temp[col]=df_temp[col]+abs(minimo)+1
    else:
        df_temp[col]=df_temp[col]

# Aplicando transformacion yeon

for c in df_temp[continuas].columns: 
    df_temp[c],_=stats.yeojohnson(df_temp[c])

In [12]:
df_temp['Credit_Score'].value_counts()

Standard    32991
Poor        26773
Good        16445
Name: Credit_Score, dtype: int64

In [13]:
# Codificacion variables categoricas

CATEGORICAL_VARS = ['SSN','Occupation','Payment_of_Min_Amount','Payment_Behaviour','Month','Credit_Mix','Credit_Score']

for c in df_temp[CATEGORICAL_VARS].columns:
    Variable_freq_mapper=(df_temp[c].value_counts().sort_values(ascending=False)).to_dict()
    df_temp[c]=df_temp[c].map(Variable_freq_mapper)

In [14]:
# Seleccionando final

df_final=df_temp.copy()[[ 
'Annual_Income',
'Interest_Rate',
'Outstanding_Debt',
'Num_Bank_Accounts',
'Num_Credit_Card',
'Num_Credit_Inquiries',
'Delay_from_due_date',
'Monthly_Balance',
'Month',
'SSN',
'Occupation',
'Credit_Mix',
'Payment_of_Min_Amount',
'Payment_Behaviour',
 'Credit_Score']
]

In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76209 entries, 79102 to 99983
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Annual_Income          76209 non-null  float64
 1   Interest_Rate          76209 non-null  float64
 2   Outstanding_Debt       76209 non-null  float64
 3   Num_Bank_Accounts      76209 non-null  float64
 4   Num_Credit_Card        76209 non-null  float64
 5   Num_Credit_Inquiries   76209 non-null  float64
 6   Delay_from_due_date    76209 non-null  float64
 7   Monthly_Balance        76209 non-null  float64
 8   Month                  76209 non-null  int64  
 9   SSN                    76209 non-null  int64  
 10  Occupation             76209 non-null  int64  
 11  Credit_Mix             76209 non-null  int64  
 12  Payment_of_Min_Amount  76209 non-null  int64  
 13  Payment_Behaviour      76209 non-null  int64  
 14  Credit_Score           76209 non-null  int64  
dty

### Construccion de modelos

In [16]:
X = df_final.drop(['Credit_Score'], axis=1)
y = df_final['Credit_Score']
X.head()

,Annual_Income,Interest_Rate,Outstanding_Debt,Num_Bank_Accounts,Num_Credit_Card,Num_Credit_Inquiries,Delay_from_due_date,Monthly_Balance,Month,SSN,Occupation,Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour
79102,13.879781,7.184026,58.541470,5.256536,3.375337,3.523152,5.054458,17.927258,9616,2,4829,15216,39229,14362
52133,11.377149,3.399717,37.041828,5.256536,3.055745,2.073700,4.197932,15.795480,9663,5,4829,25603,39229,21209
76068,13.132980,5.505791,47.697123,5.256536,3.375337,4.040135,4.326818,16.881846,9581,8,4829,15216,39229,14362
99657,14.527234,2.856283,24.264774,5.256536,3.673012,2.378805,4.616967,24.298353,9478,8,5430,25603,39229,11537
37405,12.455433,4.674039,27.604758,3.355321,2.709090,3.329015,3.911687,16.802644,9663,6,4764,25603,39229,9337


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=2023)

In [18]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

#### KNN modelo de clasificacion

In [19]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train_std , y_train)
knn_score_train=knn.score(X_train_std , y_train)
knn_score_train

c:\Users\50259\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.73791849435759

In [20]:
from sklearn.metrics import confusion_matrix

knn_preds = knn.predict(X_test_std)

cm = confusion_matrix(y_test, knn_preds)
cm_df = pd.DataFrame(cm, index = [32991,16445,26773], columns = [32991,16445,26773])
cm_df

c:\Users\50259\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


,32991,16445,26773
32991,3380,353,1219
16445,758,5752,1475
26773,1620,2235,6071


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, knn_preds))

              precision    recall  f1-score   support

       16445       0.59      0.68      0.63      4952
       26773       0.69      0.72      0.70      7985
       32991       0.69      0.61      0.65      9926

    accuracy                           0.66     22863
   macro avg       0.66      0.67      0.66     22863
weighted avg       0.67      0.66      0.66     22863



#### Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb_preds = gnb.fit(X_train_std, y_train).predict(X_test_std)

In [23]:
print(classification_report(y_test, gnb_preds))

              precision    recall  f1-score   support

       16445       0.53      0.74      0.62      4952
       26773       0.70      0.71      0.71      7985
       32991       0.71      0.56      0.63      9926

    accuracy                           0.65     22863
   macro avg       0.65      0.67      0.65     22863
weighted avg       0.67      0.65      0.65     22863



In [24]:

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=gnb, 
                 param_grid=params_NB, 
                 cv=KFold(n_splits=20, shuffle=True, random_state=2023),   
                 verbose=1, 
                 scoring='accuracy') 
gs_NB.fit(X_train_std, y_train)

gs_NB.best_params_

Fitting 20 folds for each of 100 candidates, totalling 2000 fits


{'var_smoothing': 0.0006579332246575676}

In [25]:
gnb = GaussianNB(var_smoothing= 0.0006579332246575676)
gnb_preds = gnb.fit(X_train_std, y_train).predict(X_test_std)
print(classification_report(y_test, gnb_preds))

              precision    recall  f1-score   support

       16445       0.53      0.74      0.62      4952
       26773       0.70      0.71      0.71      7985
       32991       0.71      0.56      0.63      9926

    accuracy                           0.65     22863
   macro avg       0.65      0.67      0.65     22863
weighted avg       0.67      0.65      0.65     22863



##### Regresion Logistica

In [26]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(solver='saga')
 
hyperparams_grid = {'penalty': ['l1', 'l2', 'elasticnet', None],
                                     'C': [0.01, 0.1, 1]
}

grid_search = GridSearchCV(estimator=logit, param_grid=hyperparams_grid, scoring=['roc_auc', 'accuracy'], refit='roc_auc', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']
roc_aucs = grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HyperParámetros: {grid_search.best_params_}')


c:\Users\50259\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\50259\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\50259\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1471, in fit
    raise ValueError(
ValueError: l1_ratio must be between 0 and 1; got (l1_ratio=None)

--------------------------------------------------------------------------------
60 fits failed with the following

Mejores HyperParámetros: {'C': 0.01, 'penalty': 'l1'}


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
logit = LogisticRegression(solver='saga',C= 0.01, penalty= 'l1')
logit.fit(X_train_std, y_train)

logit_preds = logit.predict(X_test_std)
logit_preds

array([26773, 32991, 32991, ..., 32991, 32991, 26773], dtype=int64)

In [28]:
print(classification_report(y_test, logit_preds))

              precision    recall  f1-score   support

       16445       0.59      0.55      0.57      4952
       26773       0.70      0.65      0.67      7985
       32991       0.60      0.66      0.63      9926

    accuracy                           0.63     22863
   macro avg       0.63      0.62      0.62     22863
weighted avg       0.63      0.63      0.63     22863



#### LDA

In [29]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(solver='lsqr')
 
hyperparams_grid = {'shrinkage': [0.01,0.1,1]
}

grid_search = GridSearchCV(estimator=lda, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']



mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')


Mejores HyperParámetros: {'shrinkage': 1} acc: 0.6310317680492337


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [30]:
lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage=1)
lda.fit(X_train_std, y_train)
lda_preds = lda.predict(X_test_std)
print(classification_report(y_test, lda_preds))

              precision    recall  f1-score   support

       16445       0.52      0.70      0.59      4952
       26773       0.70      0.70      0.70      7985
       32991       0.67      0.56      0.61      9926

    accuracy                           0.64     22863
   macro avg       0.63      0.65      0.63     22863
weighted avg       0.65      0.64      0.64     22863



#### Decision tree

In [31]:
from sklearn.tree  import DecisionTreeClassifier

t0 = time()
tree= DecisionTreeClassifier()
 
hyperparams_grid = {   "max_depth": [8,10,12,15],
                        "max_features":[14,10]
}

grid_search = GridSearchCV(estimator=tree, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

Hecho en 23.168s
Mejores HyperParámetros: {'max_depth': 10, 'max_features': 14} acc: 0.7194962733762478


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [32]:
tree = DecisionTreeClassifier(max_depth= 10, max_features=14)
tree.fit(X_train_std, y_train)
tree_preds = tree.predict(X_test_std)
print(classification_report(y_test, tree_preds))

              precision    recall  f1-score   support

       16445       0.63      0.71      0.67      4952
       26773       0.77      0.75      0.76      7985
       32991       0.74      0.71      0.72      9926

    accuracy                           0.72     22863
   macro avg       0.71      0.72      0.72     22863
weighted avg       0.73      0.72      0.72     22863



#### Ada Boost

In [33]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier() 

hyperparams_grid = {   "n_estimators": [8,10,12],
                        "learning_rate":[0.5,0.8]
}

grid_search = GridSearchCV(estimator=ada, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

Hecho en 66.172s
Mejores HyperParámetros: {'learning_rate': 0.8, 'n_estimators': 12} acc: 0.6642986120831598


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
ada = AdaBoostClassifier(learning_rate= 0.8, n_estimators= 12) 
ada.fit(X_train_std, y_train)
ada_preds = ada.predict(X_test_std)
print(classification_report(y_test, ada_preds))

              precision    recall  f1-score   support

       16445       0.58      0.68      0.63      4952
       26773       0.75      0.67      0.71      7985
       32991       0.68      0.68      0.68      9926

    accuracy                           0.68     22863
   macro avg       0.67      0.68      0.67     22863
weighted avg       0.68      0.68      0.68     22863



#### XGboost

In [35]:
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder
#este modelo necesita que la codificacion empiece en cero por lo que se realiza la transformacion:
le = LabelEncoder()
y_train2 = le.fit_transform(y_train)

xgb = XGBClassifier(objective = 'binary:logistic')

hyperparams_grid = {   "max_depth": [3,5],
                        "learning_rate":[0.5,0.3],
                        "n_estimators":[3,5],
                        "random_state":[10,42]

}

grid_search = GridSearchCV(estimator=xgb, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train2)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

Hecho en 175.415s
Mejores HyperParámetros: {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 5, 'random_state': 10} acc: 0.711349020020361


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [36]:
xgb = XGBClassifier(objective = 'binary:logistic',learning_rate= 0.5, max_depth=5, n_estimators=5, random_state= 10)
xgb.fit(X_train_std, y_train2)
xgb_preds = xgb.predict(X_test_std)
y_test2 = le.fit_transform(y_test)
print(classification_report(y_test2, xgb_preds))

              precision    recall  f1-score   support

           0       0.62      0.73      0.67      4952
           1       0.76      0.77      0.76      7985
           2       0.75      0.68      0.72      9926

    accuracy                           0.72     22863
   macro avg       0.71      0.73      0.72     22863
weighted avg       0.73      0.72      0.72     22863



#### LGBM

In [37]:
import lightgbm as lgb

lg=lgb.LGBMClassifier()

hyperparams_grid = {   "max_depth": [8,12],
                        "learning_rate":[0.5,0.8],
                        "n_estimators":[4,6],
                        "random_state":[12,42]

}

grid_search = GridSearchCV(estimator=lg, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')


Hecho en 248.771s
Mejores HyperParámetros: {'learning_rate': 0.5, 'max_depth': 8, 'n_estimators': 6, 'random_state': 12} acc: 0.7250845541796032


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [38]:
lg=lgb.LGBMClassifier(learning_rate= 0.5, max_depth= 12, n_estimators=6, random_state=12)
lg.fit(X_train_std, y_train)
lg_preds = lg.predict(X_test_std)
print(classification_report(y_test, lg_preds))

              precision    recall  f1-score   support

       16445       0.64      0.73      0.68      4952
       26773       0.76      0.77      0.77      7985
       32991       0.76      0.69      0.72      9926

    accuracy                           0.73     22863
   macro avg       0.72      0.73      0.72     22863
weighted avg       0.73      0.73      0.73     22863



#### LQDA

In [39]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

lq=QuadraticDiscriminantAnalysis()

hyperparams_grid = {   "reg_param": [0.15,0.2,0.5],
                        "store_covariance":[True,False]
}

grid_search = GridSearchCV(estimator=lq, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

Hecho en 256.624s
Mejores HyperParámetros: {'reg_param': 0.2, 'store_covariance': True} acc: 0.653901280986803


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [40]:
lq=QuadraticDiscriminantAnalysis(reg_param= 0.2, store_covariance=True)
lq.fit(X_train_std, y_train)
lq_preds = lg.predict(X_test_std)
print(classification_report(y_test, lq_preds))

              precision    recall  f1-score   support

       16445       0.64      0.73      0.68      4952
       26773       0.76      0.77      0.77      7985
       32991       0.76      0.69      0.72      9926

    accuracy                           0.73     22863
   macro avg       0.72      0.73      0.72     22863
weighted avg       0.73      0.73      0.73     22863



#### Random Forest

In [41]:
from sklearn.ensemble  import RandomForestClassifier

rf= RandomForestClassifier()
 
hyperparams_grid = {   "max_depth": [8,12],
                        "max_features":[6,10]
}

grid_search = GridSearchCV(estimator=rf, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

Hecho en 914.449s
Mejores HyperParámetros: {'max_depth': 12, 'max_features': 10} acc: 0.736816910372148


c:\Users\50259\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\50259\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [42]:
rf= RandomForestClassifier(max_depth=12, max_features=10)
rf.fit(X_train_std, y_train)
rf_preds = rf.predict(X_test_std)
print(classification_report(y_test, rf_preds))

              precision    recall  f1-score   support

       16445       0.65      0.75      0.70      4952
       26773       0.79      0.79      0.79      7985
       32991       0.77      0.72      0.75      9926

    accuracy                           0.75     22863
   macro avg       0.74      0.75      0.75     22863
weighted avg       0.75      0.75      0.75     22863



### SVM

In [43]:
from sklearn.svm  import SVC

svm= SVC()
 
hyperparams_grid = {   "shrinking": [True,False]                        
}

grid_search = GridSearchCV(estimator=svm, param_grid=hyperparams_grid, scoring=['accuracy'], refit='accuracy', n_jobs =-1, 
                                                                                     cv=KFold(n_splits=20, shuffle=True, random_state=2023))
grid_search.fit(X_train_std, y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']

mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print("Hecho en %0.3fs" % (time() - t0))
print(f'Mejores HyperParámetros: {grid_search.best_params_} acc: {np.mean([x for x in accuracies if not np.isnan(x)])}')

### Conclusiones

Por las metricas obtenidas el listado de modelos de mejor a peor se encuentra ordenado como:

1. Random Forrest 
2. LQDA
3. LGBM
4. Decision Tree
5. XGBoost
6. KNN
7. AdaBoost
8. Naive Bayes
9. LDA
10. Logistic Regression

No obstante, al momento de comparar el tiempo de entrenamiento necesario para cada modelo se opto por utilizar Decision Tree
debido a que no consume tanto tiempo ni recurso de procesamiento de la computadora en comparacion al resto de modelos y se tienen 
resultados efectivos
